# Project: Evaluate another STRIPS Pricing Scheme
In this project, students will evaluate another pricing scheme for the [Registered Interest and Principal of Securities (STRIPS)](https://www.treasurydirect.gov/marketable-securities/strips/) bonds.

## Learning objectives
Fill me in

## Setup
We set up the computational environment by including the `Include.jl` file. The `Include.jl` file loads external packages, various functions we will use in the exercise, and custom types to model the components of our example problem.
* For additional information on functions and types used in this material, see the [Julia programming language documentation](https://docs.julialang.org/en/v1/) and the [VLQuantitativeFinancePackage.jl documentation](https://github.com/varnerlab/VLQuantitativeFinancePackage.jl).

In [1]:
include("Include.jl");

    Updating git-repo `https://github.com/varnerlab/VLQuantitativeFinancePackage.jl.git`
   Resolving package versions...
  No Changes to `~/Desktop/julia_work/CHEME-130-eCornell-Repository/courses/CHEME-131/module-3/Project.toml`
  No Changes to `~/Desktop/julia_work/CHEME-130-eCornell-Repository/courses/CHEME-131/module-3/Manifest.toml`
  Activating project at `~/Desktop/julia_work/CHEME-130-eCornell-Repository/courses/CHEME-131/module-3`
  No Changes to `~/Desktop/julia_work/CHEME-130-eCornell-Repository/courses/CHEME-131/module-3/Project.toml`
  No Changes to `~/Desktop/julia_work/CHEME-130-eCornell-Repository/courses/CHEME-131/module-3/Manifest.toml`
    Updating registry at `~/.julia/registries/General.toml`
    Updating git-repo `https://github.com/varnerlab/VLQuantitativeFinancePackage.jl.git`
  No Changes to `~/Desktop/julia_work/CHEME-130-eCornell-Repository/courses/CHEME-131/module-3/Project.toml`
  No Changes to `~/Desktop/julia_work/CHEME-130-eCornell-Repository/courses/CH

## Prerequisite: Load the Treasury auction dataset
We'll explore `T-note` and `T-bond` prices from United States Treasury auctions between October 2022 and August 2023 downloaded as a `CSV` file using the [Auction query functionality of TreasuryDirect.gov](https://www.treasurydirect.gov/auctions/auction-query/). 
* We load the `CSV` dataset using the `MyTreasuryNotesAndBondsDataSet()` function, which returns the auction data as a `DataFrame`. We store the auction data in the `dataset`

In [2]:
dataset = MyTreasuryNotesAndBondsDataSet();

## Objective 1: Select a random note or bond and generate STRIPS
We'll select a random note or bond, compute its price, compare it to the auction price (to `check` our price codes), and then `strip` the selected note or bond.
* First, determine how many notes and bonds we have in the `dataset` using the [nrow function](https://dataframes.juliadata.org/stable/lib/functions/#DataAPI.nrow) that is exported by the [DataFrames.jl package](https://github.com/JuliaData/DataFrames.jl). Then, compute a random index using the [built-in Julia rand function](https://docs.julialang.org/en/v1/stdlib/Random/). Store the auction data for the random note or bond in the `random_note_or_bond` variable

In [3]:
number_of_notes_and_bonds = nrow(dataset)
selected_bond_index = rand(1:number_of_notes_and_bonds);
random_note_or_bond = dataset[selected_bond_index,:]

Row,CUSIP,Security Type,Security Term,Auction Date,Issue Date,Maturity Date,Price,High Yield,Interest Rate
,String15,String7,String7,String15,String15,String15,Float64,Float64,Float64
4,912810TU2,Bond,20-Year,08/23/2023,08/31/2023,08/15/2043,98.3742,0.04499,0.04375


Next, instantiate a discrete compounding model, and store this model in the `discrete_compounding` variable:

In [4]:
# Build a discrete compounding model -
discrete_compounding = DiscreteCompoundingModel();

Get the parameters from `random_note_or_bond` variable to test our price calculation. 
* In particular, we set the face (par) value `Vₚ,` the duration `T` (which we convert to the number of years using the `securityterm` function), the yield field, i.e., the $\bar{r}$ value in the discount rate, the coupon rate $c$ and the number of coupon payments per year `λ`.

In [5]:
Vₚ = 100.0; # face value of the note or bond
T = random_note_or_bond[Symbol("Security Term")] |> String |> securityterm; # term as a Float64
r̄ = random_note_or_bond[Symbol("High Yield")]; # Effective discount rate
c = random_note_or_bond[Symbol("Interest Rate")]; # Interest rate (coupon rate)
λ = 2; # number of coupon payments per year

Finally,  compute the price of the note (or bond) using the short-cut syntax and the discount model. We store the populated note or bond in the `model` variable, and the computed price in the `nominal_computed_price` (the `price` field on the `model`):

In [6]:
# Build a T-year model and compute the price using the short-hand syntax
model = build(MyUSTreasuryCouponSecurityModel, (
            par = Vₚ, T = T, rate = r̄, coupon = c, λ = λ
        )) |> discrete_compounding;

nominal_computed_price = model.price;

### TODO: Strip the note or bond to create a collection of zero-coupon payments
Generate a dictionary of zero-coupon bond models for a `MyUSTreasuryCouponSecurityModel` using the `strip(...)` function exported by the [VLQuantitativeFinancePackage.jl package](https://github.com/varnerlab/VLQuantitativeFinancePackage.jl) 
* The resulting `strips` dictionary stores the generated bonds in order of duration, with their indexes serving as keys and the corresponding `MyUSTreasuryZeroCouponBondModel` instances as values. 

In [7]:
strips = VLQuantitativeFinancePackage.strip(model);

## Objective 2: Evaluate new STRIPS pricing approach
Let's develop a new pricing approach for STRIPS. Suppose the yield we are willing to sell the bond is given by:
$$
\bar{r}_{i} = \theta_{1}+\theta_{2}\cdot\left(i-1\right)^{\beta}
$$
where $i$ denotes the STRIPS index, $\theta_{1}$ denotes the interest rate for the shortest term bond, $\theta_{2}$ governs how fast the rate escalates for each pass of the loop, i.e., for each bond of longer maturity that we sell, and $\beta$ is a sensitivity parameter.
* Because $\theta_{1}\geq{0}$ and $\theta_{2}\geq{0}$, this scheme will give a standard yield curve $\bar{r}_{i}\leq\bar{r}_{i+1}$, i.e., longer-term maturity will have a higher-yield compared to shorter-term bonds.

In [8]:
θ₁, θ₂, β = 0.01, 0.001, 1.0;
number_of_zc_bonds = length(strips)
for i ∈ 1:number_of_zc_bonds

    ### BEGIN SOLUTION
    zc_model = strips[i];
    zc_model.rate = θ₁ + θ₂*(i-1)^β
    zc_model = price(zc_model, discrete_compounding)
    ### END SOLUTION
end

### Visualize the first $m$ STRIPS products
`Unhide` the next code cell to see how we build a table displaying the bond price information. 
* First, specify the `number_of_strips_to_view.` Then, we'll loop through each STRIP and access the respective models from the `strips` dictionary using the bracket notation `strips[i],` where `i` represents the STRIP index. 
* Next, we'll add pricing, maturity, etc. data to the `strips_table_data_array` and display the resulting table by calling the `pretty_table(…)` function, which is exported by the [PrettyTables.jl package](https://github.com/ronisbr/PrettyTables.jl)

In [9]:
number_of_strips = length(strips);
number_of_strips_to_view = number_of_strips ≥ 10 ? 10 : number_of_strips
strips_table_data_array = Array{Any,2}(undef, number_of_strips, 6);
strips_table_header_array = (["", "Maturity","Face","Price","r̄", "Discount"],
    ["","YR","USD","USD","", "price/face"]);
for i ∈ 1:number_of_strips

    zc_model = strips[i];
    strips_table_data_array[i,1] = i
    strips_table_data_array[i,2] = zc_model.T
    strips_table_data_array[i,3] = zc_model.par
    strips_table_data_array[i,4] = zc_model.price
    strips_table_data_array[i,5] = zc_model.rate  
    strips_table_data_array[i,6] = (zc_model.price/zc_model.par)
end
pretty_table(strips_table_data_array[1:number_of_strips_to_view,1:end], 
    header=strips_table_header_array, tf = tf_simple)

===== ========== ======== ========= ======= =============
       Maturity     Face     Price       r̄     Discount 
             YR      USD       USD           price/face 
===== ========== ======== ========= ======= =============
   1        0.5   2.1875   2.17662    0.01     0.995025
   2        1.0   2.1875   2.15505   0.015     0.985167
   3        1.5   2.1875   2.12317    0.02      0.97059
   4        2.0   2.1875   2.08146   0.025     0.951524
   5        2.5   2.1875   2.03057    0.03      0.92826
   6        3.0   2.1875   1.97125   0.035     0.901143
   7        3.5   2.1875   1.90435    0.04      0.87056
   8        4.0   2.1875    1.8308   0.045     0.836938
   9        4.5   2.1875   1.75159    0.05     0.800728
  10        5.0   2.1875   1.66775   0.055     0.762398
===== ========== ======== ========= ======= =============


#### Check: How much do we collect from the sale of the STRIPS?
In this scenario, we purchased an original instrument from the [United States Treasury](https://www.treasurydirect.gov/) for $V_{B}$, stripped the coupons and face, and resold them. How much did we make from the sale of the STRIPS?

In [10]:
resale_total = sum(strips_table_data_array[:,4]);
println("Purchased bond $(selected_bond_index) for $(round(nominal_computed_price,digits=3)) USD, stripped and resold it for $(round(resale_total, digits=3)) USD")

Purchased bond 4 for 98.376 USD, stripped and resold it for 38.818 USD


## Disclaimer and Risks
__This content is offered solely for training and informational purposes__. No offer or solicitation to buy or sell securities or derivative products or any investment or trading advice or strategy is made, given, or endorsed by the teaching team. 

__Trading involves risk__. Carefully review your financial situation before investing in securities, futures contracts, options, or commodity interests. Past performance, whether actual or indicated by historical tests of strategies, is no guarantee of future performance or success. Trading is generally inappropriate for someone with limited resources, investment or trading experience, or a low-risk tolerance.  Only risk capital that is not required for living expenses.

__You are fully responsible for any investment or trading decisions you make__. Such decisions should be based solely on evaluating your financial circumstances, investment or trading objectives, risk tolerance, and liquidity needs.